<a href="https://colab.research.google.com/github/CalvHobbes/ai-agents/blob/main/Gemini_Agents.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Code based on [Google Agents whitepaper](https://drive.google.com/file/d/1oEjiRCTbd54aSdB_eEe3UShxLBWK9xkt/view), except Vertex replaced by Gemini (so no billing setup required)

**FUNCTIONS**

In [1]:
from typing import Optional

def display_cities(cities: list[str], preferences: Optional[str] = None):
  """Provides a list of cities based on the user's search query and preferences.
  Args:
  preferences (str): The user's preferences for the search, like skiing,
  beach, restaurants, bbq, etc.
  cities (list[str]): The list of cities being recommended to the user.
  Returns:
  list[str]: The list of cities being recommended to the user.
  """
  return cities

In [ ]:
!pip install google-cloud-aiplatform google-auth


In [3]:
from google.colab import userdata
import os

os.environ["GOOGLE_API_KEY"] = userdata.get('GEMINI_KEY')
os.environ["GOOGLE_CLOUD_PROJECT"] = userdata.get('GOOGLE_CLOUD_PROJECT') # Replace with your actual project ID
# print(os.environ["GOOGLE_API_KEY"])
# print(os.environ["GOOGLE_CLOUD_PROJECT"] )


In [ ]:
%pip install -U google-generativeai

In [5]:
import os
import google.generativeai as genai

genai.configure(api_key=os.environ['GOOGLE_API_KEY'])

In [6]:
model = genai.GenerativeModel("gemini-1.5-flash-001")

In [8]:
from google.generativeai.types import FunctionDeclaration, Tool

# display_cities_function = FunctionDeclaration.from_func(display_cities) // For Vertex
# Define FunctionDeclaration manually
display_cities_function = FunctionDeclaration(
    name="display_cities",
    description="Provides a list of cities based on the user's search query and preferences.",
    parameters={
        "type": "object",
        "properties": {
            "cities": {
                "type": "array",
                "items": {"type": "string"},
                "description": "The list of cities being recommended to the user."
            },
            "preferences": {
                "type": "string",
                "description": "The user's preferences for the search, like skiing, beach, restaurants, bbq, etc."
            }
        },
        "required": ["cities"]
    }
)
tool = Tool(function_declarations=[display_cities_function])
message = "I’d like to take a ski trip with my family but I’m not sure where to go."

In [9]:
res = model.generate_content(message, tools=[tool])
print(f"Function Name: {res.candidates[0].content.parts[0].function_call.name}")
function_args = res.candidates[0].content.parts[0].function_call.args
# Convert the MapComposite object to a dictionary
function_args_dict = dict(function_args)

print(f"Function Args: {function_args_dict}")

Function Name: display_cities
Function Args: {'preferences': 'skiing', 'cities': ['Aspen', 'Vail', 'Breckenridge']}
